## Изучение кластеризации методом K-Means с использованием R и принципов "tidy data"

### [**Тест перед лекцией**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

В этом уроке вы научитесь создавать кластеры с использованием пакета Tidymodels и других пакетов из экосистемы R (назовем их друзьями 🧑‍🤝‍🧑), а также набора данных о нигерийской музыке, который вы импортировали ранее. Мы рассмотрим основы метода K-Means для кластеризации. Помните, как вы узнали в предыдущем уроке, существует множество способов работы с кластерами, и выбор метода зависит от ваших данных. Мы попробуем метод K-Means, так как это наиболее распространенная техника кластеризации. Давайте начнем!

Термины, с которыми вы познакомитесь:

- Оценка силуэта (Silhouette scoring)

- Метод локтя (Elbow method)

- Инерция (Inertia)

- Дисперсия (Variance)

### **Введение**

[Кластеризация методом K-Means](https://wikipedia.org/wiki/K-means_clustering) — это метод, происходящий из области обработки сигналов. Он используется для разделения и группировки данных на `k кластеров` на основе сходства их характеристик.

Кластеры можно визуализировать как [диаграммы Вороного](https://wikipedia.org/wiki/Voronoi_diagram), которые включают точку (или "семя") и соответствующую ей область.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Инфографика от Джен Лупер</figcaption>

Метод K-Means включает следующие шаги:

1. Дата-сайентист начинает с указания желаемого количества кластеров, которые нужно создать.

2. Затем алгоритм случайным образом выбирает K наблюдений из набора данных, чтобы использовать их в качестве начальных центров кластеров (т.е. центроидов).

3. Далее каждое из оставшихся наблюдений назначается к своему ближайшему центроиду.

4. Затем вычисляется новое среднее значение для каждого кластера, и центроид перемещается к этому среднему.

5. После пересчета центров каждое наблюдение снова проверяется, чтобы определить, не стало ли оно ближе к другому кластеру. Все объекты перераспределяются с использованием обновленных средних значений кластеров. Шаги перераспределения кластеров и обновления центроидов повторяются итеративно до тех пор, пока распределение кластеров не перестанет изменяться (т.е. пока не будет достигнута сходимость). Обычно алгоритм завершается, когда каждая новая итерация приводит к незначительному перемещению центроидов, и кластеры становятся статичными.

<div>

> Обратите внимание, что из-за случайного выбора начальных k наблюдений, используемых в качестве стартовых центроидов, мы можем получать немного разные результаты при каждом применении процедуры. По этой причине большинство алгоритмов используют несколько *случайных запусков* и выбирают итерацию с наименьшим WCSS. Таким образом, настоятельно рекомендуется всегда запускать K-Means с несколькими значениями *nstart*, чтобы избежать *нежелательного локального оптимума.*

</div>

Эта короткая анимация с использованием [иллюстраций](https://github.com/allisonhorst/stats-illustrations) Эллисон Хорст объясняет процесс кластеризации:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Иллюстрация от @allison_horst</figcaption>

Основной вопрос, который возникает при кластеризации, заключается в следующем: как определить, на сколько кластеров разделить ваши данные? Один из недостатков метода K-Means заключается в том, что вам нужно заранее установить `k`, то есть количество `центроидов`. К счастью, `метод локтя` помогает оценить хорошее начальное значение для `k`. Вы попробуете это через минуту.

### 

**Предварительные требования**

Мы начнем с того места, где остановились в [предыдущем уроке](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), где мы анализировали набор данных, создавали множество визуализаций и фильтровали данные для интересующих нас наблюдений. Обязательно ознакомьтесь с ним!

Нам понадобятся некоторые пакеты для выполнения этого модуля. Вы можете установить их с помощью команды: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Или используйте следующий скрипт, который проверяет наличие необходимых пакетов и устанавливает их, если каких-то не хватает.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Давайте начнем!

## 1. Танец с данными: Выберите 3 самых популярных музыкальных жанра

Это краткое напоминание о том, что мы делали в предыдущем уроке. Давайте разберем данные!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Это прошло отлично!

## 2. Дополнительное исследование данных.

Насколько чисты эти данные? Давайте проверим наличие выбросов с помощью ящиков с усами (boxplots). Мы сосредоточимся на числовых столбцах с меньшим количеством выбросов (хотя вы можете удалить выбросы). Ящики с усами могут показать диапазон данных и помогут выбрать, какие столбцы использовать. Обратите внимание, что ящики с усами не показывают дисперсию, которая является важным элементом для хорошо кластеризуемых данных. Пожалуйста, ознакомьтесь с [этим обсуждением](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) для получения дополнительной информации.

[Ящики с усами](https://en.wikipedia.org/wiki/Box_plot) используются для графического отображения распределения `числовых` данных, поэтому давайте начнем с *выбора* всех числовых столбцов вместе с популярными музыкальными жанрами.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Посмотрите, как удобно использовать помощник выбора `where` 💁? Исследуйте другие подобные функции [здесь](https://tidyselect.r-lib.org/).

Поскольку мы будем создавать ящик с усами для каждой числовой переменной и хотим избежать использования циклов, давайте преобразуем наши данные в *длинный* формат, который позволит нам воспользоваться `facets` - подграфиками, каждый из которых отображает один поднабор данных.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Теперь намного длиннее! А теперь время для `ggplots`! Какой `geom` мы будем использовать?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Легко-gg!

Теперь мы видим, что данные немного шумные: наблюдая каждый столбец в виде коробчатой диаграммы, можно заметить выбросы. Вы могли бы пройтись по набору данных и удалить эти выбросы, но тогда данные станут довольно минимальными.

На данный момент давайте выберем, какие столбцы будем использовать для нашего упражнения по кластеризации. Выберем числовые столбцы с похожими диапазонами. Мы могли бы закодировать `artist_top_genre` как числовой, но пока исключим его.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Вычисление кластеризации методом k-средних в R

Мы можем вычислить метод k-средних в R с помощью встроенной функции `kmeans`, см. `help("kmeans()")`. Функция `kmeans()` принимает на вход датафрейм, содержащий только числовые столбцы, в качестве основного аргумента.

Первым шагом при использовании кластеризации методом k-средних является указание количества кластеров (k), которые будут созданы в итоговом решении. Мы знаем, что в наборе данных есть 3 музыкальных жанра, которые мы выделили, поэтому попробуем указать 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Объект kmeans содержит несколько частей информации, которые хорошо объяснены в `help("kmeans()")`. Сейчас давайте сосредоточимся на некоторых из них. Мы видим, что данные были сгруппированы в 3 кластера размером 65, 110 и 111. Вывод также содержит центры кластеров (средние значения) для 3 групп по 5 переменным.

Вектор кластеризации — это назначение кластера для каждой наблюдаемой точки. Давайте используем функцию `augment`, чтобы добавить назначение кластера в исходный набор данных.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Отлично, мы только что разделили наш набор данных на 3 группы. Итак, насколько хороша наша кластеризация 🤷? Давайте посмотрим на `Silhouette score`.

### **Silhouette score**

[Анализ силуэта](https://en.wikipedia.org/wiki/Silhouette_(clustering)) может быть использован для изучения расстояния между получившимися кластерами. Этот показатель варьируется от -1 до 1, и если значение близко к 1, кластер плотный и хорошо отделен от других кластеров. Значение, близкое к 0, указывает на пересекающиеся кластеры, где образцы находятся очень близко к границе разделения соседних кластеров. [Источник](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Метод среднего силуэта вычисляет средний силуэт наблюдений для различных значений *k*. Высокий средний показатель силуэта указывает на хорошую кластеризацию.

Функция `silhouette` из пакета cluster используется для вычисления средней ширины силуэта.

> Силуэт можно рассчитать с использованием любой [метрики расстояния](https://en.wikipedia.org/wiki/Distance "Distance"), такой как [евклидово расстояние](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") или [манхэттенское расстояние](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), которые мы обсуждали в [предыдущем уроке](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Наш результат — **0.549**, то есть прямо посередине. Это указывает на то, что наши данные не особенно хорошо подходят для такого типа кластеризации. Давайте проверим эту догадку визуально. Пакет [factoextra](https://rpkgs.datanovia.com/factoextra/index.html) предоставляет функции (`fviz_cluster()`) для визуализации кластеризации.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Перекрытие кластеров указывает на то, что наши данные не очень хорошо подходят для такого типа кластеризации, но давайте продолжим.

## 4. Определение оптимального количества кластеров

Основной вопрос, который часто возникает при кластеризации методом K-Means, заключается в следующем: если у нас нет известных меток классов, как определить, на сколько кластеров нужно разделить данные?

Один из способов выяснить это — использовать выборку данных, чтобы `создать серию моделей кластеризации` с увеличивающимся количеством кластеров (например, от 1 до 10) и оценить метрики кластеризации, такие как **силуэтный коэффициент**.

Давайте определим оптимальное количество кластеров, вычислив алгоритм кластеризации для различных значений *k* и оценив **сумму квадратов внутри кластеров** (WCSS). Общая сумма квадратов внутри кластеров (WCSS) измеряет компактность кластеризации, и мы хотим, чтобы она была как можно меньше, так как более низкие значения означают, что точки данных находятся ближе друг к другу.

Давайте исследуем влияние различных значений `k` от 1 до 10 на эту кластеризацию.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Теперь, когда у нас есть общая сумма квадратов внутри кластеров (tot.withinss) для каждого алгоритма кластеризации с центром *k*, мы используем [метод локтя](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), чтобы определить оптимальное количество кластеров. Этот метод заключается в построении графика WCSS в зависимости от количества кластеров и выборе [точки перегиба кривой](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") в качестве количества кластеров для использования.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


График показывает значительное снижение WCSS (то есть большую *сжатость*) при увеличении количества кластеров с одного до двух, а затем еще одно заметное снижение с двух до трех кластеров. После этого снижение становится менее выраженным, что приводит к появлению `локтя` 💪 на графике примерно при трех кластерах. Это хороший показатель того, что существует два-три достаточно хорошо разделенных кластера точек данных.

Теперь мы можем извлечь модель кластеризации, где `k = 3`:

> `pull()`: используется для извлечения одного столбца
>
> `pluck()`: используется для индексации структур данных, таких как списки


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Отлично! Давайте визуализируем полученные кластеры. Хотите добавить немного интерактивности с помощью `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Возможно, мы ожидали, что каждый кластер (представленный разными цветами) будет иметь четко различимые жанры (представленные разными формами).

Давайте посмотрим на точность модели.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Точность этой модели неплохая, но и не выдающаяся. Возможно, данные не подходят для кластеризации методом K-Means. Эти данные слишком несбалансированы, слабо коррелируют и имеют слишком большую дисперсию между значениями столбцов, чтобы хорошо кластеризоваться. На самом деле, формирующиеся кластеры, вероятно, сильно зависят или искажены тремя категориями жанров, которые мы определили выше.

Тем не менее, это был весьма познавательный процесс!

В документации Scikit-learn можно увидеть, что модель, подобная этой, с плохо выраженными кластерами, имеет проблему "дисперсии":

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Инфографика из Scikit-learn</figcaption>



## **Дисперсия**

Дисперсия определяется как "среднее значение квадратов отклонений от среднего" [источник](https://www.mathsisfun.com/data/standard-deviation.html). В контексте этой задачи кластеризации это означает, что значения в нашем наборе данных имеют тенденцию слишком сильно отклоняться от среднего.

✅ Это отличный момент, чтобы подумать о всех способах исправления этой проблемы. Немного подкорректировать данные? Использовать другие столбцы? Применить другой алгоритм? Подсказка: попробуйте [масштабировать данные](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/), чтобы нормализовать их, и протестируйте другие столбцы.

> Попробуйте этот '[калькулятор дисперсии](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)', чтобы лучше понять концепцию.

------------------------------------------------------------------------

## **🚀Задание**

Поработайте с этой тетрадью, изменяя параметры. Можете ли вы улучшить точность модели, очистив данные (например, удалив выбросы)? Вы можете использовать веса, чтобы придать больший вес определенным образцам данных. Что еще можно сделать, чтобы создать более качественные кластеры?

Подсказка: попробуйте масштабировать данные. В тетради есть закомментированный код, который добавляет стандартное масштабирование, чтобы столбцы данных стали более похожими друг на друга по диапазону. Вы заметите, что, хотя коэффициент силуэта снижается, "излом" на графике локтя становится более плавным. Это связано с тем, что оставление данных без масштабирования позволяет данным с меньшей дисперсией иметь больший вес. Подробнее об этой проблеме можно прочитать [здесь](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Тест после лекции**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Обзор и самостоятельное изучение**

-   Ознакомьтесь с симулятором K-Means [например, с этим](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Вы можете использовать этот инструмент для визуализации образцов данных и определения их центроидов. Вы можете редактировать случайность данных, количество кластеров и центроидов. Помогает ли это лучше понять, как данные могут быть сгруппированы?

-   Также посмотрите [этот материал о K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) от Стэнфорда.

Хотите попробовать свои новые навыки кластеризации на наборах данных, которые хорошо подходят для метода K-Means? Ознакомьтесь с:

-   [Обучение и оценка моделей кластеризации](https://rpubs.com/eR_ic/clustering) с использованием Tidymodels и других инструментов

-   [Анализ кластеров методом K-Means](https://uc-r.github.io/kmeans_clustering), Руководство по программированию на R для бизнес-аналитики UC

- [Кластеризация методом K-Means с использованием принципов работы с tidy-данными](https://www.tidymodels.org/learn/statistics/k-means/)

## **Задание**

[Попробуйте разные методы кластеризации](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## СПАСИБО:

[Джен Лупер](https://www.twitter.com/jenlooper) за создание оригинальной версии этого модуля на Python ♥️

[`Эллисон Хорст`](https://twitter.com/allison_horst/) за создание потрясающих иллюстраций, которые делают R более доступным и увлекательным. Найдите больше иллюстраций в ее [галерее](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Приятного обучения,

[Эрик](https://twitter.com/ericntay), Золотой посол Microsoft Learn Student.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Иллюстрация от @allison_horst</figcaption>



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникающие в результате использования данного перевода.
